In [ ]:
#Testing models

In [1]:
# -*- coding: UTF-8 -*-
# Local modules
import os
import sys
import argparse
# 3rd-Party Modules
import numpy as np
import pickle as pk
import pandas as pd
from tqdm import tqdm
import glob
import librosa
import copy

# PyTorch Modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import ConcatDataset, DataLoader
import torch.optim as optim
from transformers import AutoModel
import importlib
# Self-Written Modules
sys.path.append(os.getcwd())
sys.path.append('../')
from benchmark import net
from benchmark import utils

In [2]:
import logging

In [ ]:
# !pip install speechbrain
# !pip install torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
# !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!pip uninstall torchaudio -y

In [ ]:
!pip install torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116

In [3]:
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder


In [4]:
df = pd.read_csv('../benchmark/processed_labels.csv')
df.head(2)

,FileName,Angry,Sad,Happy,Surprise,Fear,Disgust,Contempt,Neutral,Split_Set
0,MSP-PODCAST_0002_0033.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development
1,MSP-PODCAST_0002_0039.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development


In [10]:
signal, fs = torchaudio.load(f"../data/Audios/{df.FileName.values[0]}")
embeddings = classifier.encode_batch(signal)

In [11]:
embeddings.shape

torch.Size([1, 1, 512])

In [30]:
import librosa
signal, fs = librosa.load(f"../data/Audios/{df.FileName.values[0]}", sr = 16000)
embeddings2 = classifier.encode_batch(torch.FloatTensor(signal))

In [31]:
embeddings2.shape

torch.Size([1, 1, 512])

In [32]:
embeddings[0,0,:5]

tensor([-22.9484, -12.4350,   8.0856,   4.4723, -17.4227])

In [33]:
embeddings2[0,0,:5]

tensor([-22.9484, -12.4350,   8.0855,   4.4724, -17.4227])

In [5]:
!ls ../data/

Audios		  Labels	  Speaker_ids.txt  readme.txt
Audios.tar.gz	  Labels.zip	  Transcripts
ForceAligned.zip  Partitions.txt  Transcripts.zip


In [6]:
import os

os.listdir(f"../data/Audios/{df.FileName.values[0]}")[:10]

NotADirectoryError: [Errno 20] Not a directory: '../data/Audios/MSP-PODCAST_0002_0033.wav'

In [8]:
import os

os.listdir(f"../data/Audios/")[:10]

['MSP-PODCAST_3085_0127.wav',
 'MSP-PODCAST_5474_0045.wav',
 'MSP-PODCAST_2505_0470.wav',
 'MSP-PODCAST_1698_0349.wav',
 'MSP-PODCAST_2214_0291.wav',
 'MSP-PODCAST_1133_0026_0025.wav',
 'MSP-PODCAST_4748_0103_0004.wav',
 'MSP-PODCAST_4301_0026_0004.wav',
 'MSP-PODCAST_3002_0039.wav',
 'MSP-PODCAST_0642_0144.wav']